In [1]:
# Import our dependencies
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import datetime
import re
import csv
import json
import os
# Read in "The IMbd Movies dataset"
# if this is read in as csv, each row is a string and dictionaries are not recognized.
IMDb_movies_df = pd.read_csv("Resources/IMDb movies.csv", low_memory=False)
IMDb_movies_df.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [2]:
print(IMDb_movies_df.describe())
print(IMDb_movies_df.columns)
#print(movies_df["release_date"].dtypes)

           duration      avg_vote         votes     metascore  \
count  85855.000000  85855.000000  8.585500e+04  13305.000000   
mean     100.351418      5.898656  9.493490e+03     55.896881   
std       22.553848      1.234987  5.357436e+04     17.784874   
min       41.000000      1.000000  9.900000e+01      1.000000   
25%       88.000000      5.200000  2.050000e+02     43.000000   
50%       96.000000      6.100000  4.840000e+02     57.000000   
75%      108.000000      6.800000  1.766500e+03     69.000000   
max      808.000000      9.900000  2.278845e+06    100.000000   

       reviews_from_users  reviews_from_critics  
count        78258.000000          74058.000000  
mean            46.040826             27.479989  
std            178.511411             58.339158  
min              1.000000              1.000000  
25%              4.000000              3.000000  
50%              9.000000              8.000000  
75%             27.000000             23.000000  
max          1

In [3]:
print(IMDb_movies_df["date_published"].dtypes)

object


In [4]:
print(IMDb_movies_df["year"].dtypes)

object


In [5]:
IMDb_movies_df.rename(columns={'imdb_title_id':'imdb_id'}, inplace=True)
IMDb_movies_df.head()

,imdb_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [6]:
#IMDb_movies_df = IMDb_movies_df.drop(IMDb_movies_df[(IMDb_movies_df.country != 'USA') | (IMDb_movies_df.country != 'US')].index)
# IMDb_movies_df = IMDb_movies_df.drop(IMDb_movies_df[IMDb_movies_df.country != 'USA'].index)
print(IMDb_movies_df.describe())

           duration      avg_vote         votes     metascore  \
count  85855.000000  85855.000000  8.585500e+04  13305.000000   
mean     100.351418      5.898656  9.493490e+03     55.896881   
std       22.553848      1.234987  5.357436e+04     17.784874   
min       41.000000      1.000000  9.900000e+01      1.000000   
25%       88.000000      5.200000  2.050000e+02     43.000000   
50%       96.000000      6.100000  4.840000e+02     57.000000   
75%      108.000000      6.800000  1.766500e+03     69.000000   
max      808.000000      9.900000  2.278845e+06    100.000000   

       reviews_from_users  reviews_from_critics  
count        78258.000000          74058.000000  
mean            46.040826             27.479989  
std            178.511411             58.339158  
min              1.000000              1.000000  
25%              4.000000              3.000000  
50%              9.000000              8.000000  
75%             27.000000             23.000000  
max          1

In [7]:
# clean up and change date_published to datetime
# save at each change to make reverting easy
date_p=IMDb_movies_df["date_published"]
date_p.str.strip()  #remove spaces
form1=r'(^[0-9]{4})'
rel_year = date_p.str.extract(f'({form1})')  #extract just year
rel_year["release_year"]=pd.to_numeric(rel_year[0])
print(rel_year.dtypes)
rel_year.drop([0,1],axis=1,inplace=True)
print(f'null rows: {rel_year.isnull().sum()}')
rel_year.dropna(how="any", inplace=True)
print(f'number of movies: {len(rel_year)}')
print(rel_year.dtypes)

0                object
1                object
release_year    float64
dtype: object
null rows: release_year    1
dtype: int64
number of movies: 85854
release_year    float64
dtype: object


In [8]:
# rel_year=rel_year[rel_year["release_year"] > 1989]
rel_year=rel_year[rel_year["release_year"] > 1969]
year_counts=rel_year["release_year"].value_counts()
year_counts

2018.0    3364
2017.0    3355
2016.0    3153
2019.0    3120
2015.0    2971
2014.0    2909
2013.0    2771
2012.0    2492
2011.0    2419
2009.0    2252
2010.0    2248
2008.0    2207
2007.0    2066
2006.0    2028
2005.0    1830
2004.0    1636
2003.0    1535
2002.0    1365
2000.0    1360
2001.0    1352
1999.0    1346
2020.0    1339
1998.0    1030
1997.0    1011
1996.0     949
1989.0     917
1995.0     908
1988.0     904
1993.0     865
1987.0     821
1994.0     813
1991.0     791
1992.0     791
1990.0     788
1986.0     754
1982.0     679
1973.0     667
1984.0     661
1972.0     661
1985.0     656
1981.0     643
1983.0     641
1976.0     636
1971.0     623
1979.0     620
1977.0     615
1974.0     609
1975.0     595
1980.0     588
1970.0     586
1978.0     582
2021.0       7
Name: release_year, dtype: int64

In [9]:
len(rel_year)

70529

In [10]:
# awards_df = pd.read_csv("awards.csv", low_memory=False)
# awards_df.rename(columns={'const':'imdb_id'}, inplace=True)
# awards_df.head()

In [11]:
IMDb_movies_df[IMDb_movies_df['imdb_id'] == 'tt0101428']

,imdb_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
25767,tt0101428,La bella scontrosa,La Belle Noiseuse,1991,1991-09-04,Drama,238,"France, Switzerland","French, English, Hebrew",Jacques Rivette,...,"Michel Piccoli, Jane Birkin, Emmanuelle Béart,...",The former famous painter Frenhofer revisits a...,7.7,6820,NaN,$ 403056,$ 403056,NaN,55.0,43.0


In [12]:
IMDb_gt1969_df = pd.concat([IMDb_movies_df, rel_year], axis=1, ignore_index=False, join="inner")
len(IMDb_gt1969_df)

70529

In [13]:
print(IMDb_gt1969_df.columns)

Index(['imdb_id', 'title', 'original_title', 'year', 'date_published', 'genre',
       'duration', 'country', 'language', 'director', 'writer',
       'production_company', 'actors', 'description', 'avg_vote', 'votes',
       'budget', 'usa_gross_income', 'worlwide_gross_income', 'metascore',
       'reviews_from_users', 'reviews_from_critics', 'release_year'],
      dtype='object')


In [14]:
IMDb_gt1969_df.describe()

,duration,avg_vote,votes,metascore,reviews_from_users,reviews_from_critics,release_year
count,70529.000000,70529.000000,7.052900e+04,12858.000000,63624.000000,60360.000000,70529.000000
mean,102.113414,5.790186,1.093330e+04,55.158812,50.771376,30.539231,2003.242170
std,21.708421,1.275937,5.836995e+04,17.429233,195.785529,63.338663,13.620838
min,41.000000,1.000000,9.900000e+01,1.000000,1.000000,1.000000,1970.000000
25%,90.000000,5.000000,2.120000e+02,43.000000,3.000000,3.000000,1995.000000
50%,97.000000,6.000000,5.170000e+02,56.000000,9.000000,9.000000,2007.000000
75%,110.000000,6.700000,2.071000e+03,68.000000,27.000000,26.000000,2014.000000
max,808.000000,9.900000,2.278845e+06,100.000000,10472.000000,999.000000,2021.000000


In [15]:
IMDb_gt1969_df.isnull().sum()

imdb_id                      0
title                        0
original_title               0
year                         0
date_published               0
genre                        0
duration                     0
country                     64
language                   616
director                    84
writer                    1477
production_company        4229
actors                      63
description               1914
avg_vote                     0
votes                        0
budget                   49422
usa_gross_income         55897
worlwide_gross_income    40154
metascore                57671
reviews_from_users        6905
reviews_from_critics     10169
release_year                 0
dtype: int64

In [16]:
print(IMDb_gt1969_df.dtypes)

imdb_id                   object
title                     object
original_title            object
year                      object
date_published            object
genre                     object
duration                   int64
country                   object
language                  object
director                  object
writer                    object
production_company        object
actors                    object
description               object
avg_vote                 float64
votes                      int64
budget                    object
usa_gross_income          object
worlwide_gross_income     object
metascore                float64
reviews_from_users       float64
reviews_from_critics     float64
release_year             float64
dtype: object


In [17]:
df=pd.read_csv("Resources/movies_metadata.csv", low_memory=False)
print(df.describe())
print(df.columns)
print(df["release_date"].dtypes)

            revenue       runtime  vote_average    vote_count
count  4.546000e+04  45203.000000  45460.000000  45460.000000
mean   1.120935e+07     94.128199      5.618207    109.897338
std    6.433225e+07     38.407810      1.924216    491.310374
min    0.000000e+00      0.000000      0.000000      0.000000
25%    0.000000e+00     85.000000      5.000000      3.000000
50%    0.000000e+00     95.000000      6.000000     10.000000
75%    0.000000e+00    107.000000      6.800000     34.000000
max    2.787965e+09   1256.000000     10.000000  14075.000000
Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')
object


In [18]:
df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [19]:
# clean up and change release_date to datetime
# save at each change to make reverting easy
r_date=df["release_date"]
r_date.str.strip()  #remove spaces
form1=r'(^[0-9]{4})'
rel_year2 = r_date.str.extract(f'({form1})')  #extract just year
rel_year2["release_year"]=pd.to_numeric(rel_year2[0])
print(rel_year2.dtypes)
rel_year2.drop([0,1],axis=1,inplace=True)
print(f'null rows: {rel_year2.isnull().sum()}')
rel_year2.dropna(how="any", inplace=True)
print(f'number of movies: {len(rel_year2)}')
print(rel_year2.dtypes)

0                object
1                object
release_year    float64
dtype: object
null rows: release_year    90
dtype: int64
number of movies: 45376
release_year    float64
dtype: object


In [20]:
# rel_year2=rel_year2[rel_year2["release_year"] > 1989]
rel_year2=rel_year2[rel_year2["release_year"] > 1969]
year_counts2=rel_year2["release_year"].value_counts()
year_counts2

2014.0    1974
2015.0    1905
2013.0    1889
2012.0    1722
2011.0    1667
2016.0    1604
2009.0    1586
2010.0    1501
2008.0    1473
2007.0    1320
2006.0    1270
2005.0    1125
2004.0     992
2002.0     905
2003.0     882
2001.0     865
2000.0     789
1999.0     723
1998.0     722
1997.0     661
1996.0     633
1995.0     599
1994.0     544
2017.0     532
1993.0     489
1988.0     467
1987.0     462
1992.0     453
1989.0     439
1990.0     427
1991.0     426
1986.0     391
1972.0     381
1971.0     378
1982.0     368
1985.0     368
1984.0     362
1980.0     361
1981.0     360
1973.0     356
1983.0     353
1970.0     351
1974.0     348
1979.0     338
1977.0     334
1976.0     333
1975.0     332
1978.0     321
2018.0       5
2020.0       1
Name: release_year, dtype: int64

In [21]:
# # production_countries is a string that looks like a list of dictionaries
# # filter rows to keep only those with 'iso_3166_1' = 'US'
# pd_countries=df["production_countries"]  # pd_countries is a series
# form1=r"'iso_3166_1':\s*('US')"
# country = pd_countries.str.extract(f'({form1})')
# # country.drop([0],axis=1,inplace=True)
# # print(country.notnull().sum())
# # country.dropna(how="any", inplace=True)
# len(country)

In [22]:
movies_year_df = pd.concat([df, rel_year2], axis=1, ignore_index=False, join="inner")
len(movies_year_df)

37087

In [23]:
print(movies_year_df.columns)

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'release_year'],
      dtype='object')


In [24]:
print(movies_year_df.describe())

            revenue       runtime  vote_average    vote_count  release_year
count  3.708700e+04  36871.000000  37087.000000  37087.000000  37087.000000
mean   1.352415e+07     95.602587      5.632022    129.251112   2001.360935
std    7.080708e+07     38.983649      1.866179    539.555325     12.783023
min    0.000000e+00      0.000000      0.000000      0.000000   1970.000000
25%    0.000000e+00     87.000000      5.000000      4.000000   1994.000000
50%    0.000000e+00     95.000000      6.000000     11.000000   2005.000000
75%    0.000000e+00    107.000000      6.800000     43.000000   2012.000000
max    2.787965e+09   1256.000000     10.000000  14075.000000   2020.000000


In [25]:
movies_year_df.isnull().sum()

adult                        0
belongs_to_collection    33168
budget                       0
genres                       0
homepage                 29407
id                           0
imdb_id                     14
original_language            6
original_title               0
overview                   819
popularity                   0
poster_path                288
production_companies         0
production_countries         0
release_date                 0
revenue                      0
runtime                    216
spoken_languages             0
status                      78
tagline                  20712
title                        0
video                        0
vote_average                 0
vote_count                   0
release_year                 0
dtype: int64

In [26]:
# Drop some unneeded columns
# Drop missing imdb_ids (n=14)
movies_year_df.drop(["adult", "overview", "poster_path", "tagline", "video", "original_title"], axis=1, inplace=True)
movies_year_df.dropna(subset=["imdb_id"],inplace=True)
movies_year_df["imdb_id_str"]=movies_year_df["imdb_id"].apply(lambda z: z.strip())
print(movies_year_df.columns)


Index(['belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'title', 'vote_average', 'vote_count',
       'release_year', 'imdb_id_str'],
      dtype='object')


In [27]:
# Identify duplicate imdb_ids to dedupe before merging with IMDB dataset
imdb_id_str=movies_year_df["imdb_id_str"].duplicated(keep=False)
print(type(imdb_id_str))
imdb_id_str=imdb_id_str[imdb_id_str == True]

<class 'pandas.core.series.Series'>


In [28]:
# Find the duplicate imdb_ids in the dataset
# Merge duplicate imdb_ids back to dataset on index to identify which of the duplicates to keep
dupes1 = pd.merge(movies_year_df, imdb_id_str, how="inner", left_index=True, right_index=True, suffixes=['_OR','_DP'], indicator=True, validate="1:1")
print(dupes1.columns)


Index(['belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'title', 'vote_average', 'vote_count',
       'release_year', 'imdb_id_str_OR', 'imdb_id_str_DP', '_merge'],
      dtype='object')


In [29]:
# Drop colummns created in prior merge that break the following merge
dupes1.drop(["_merge", "imdb_id_str_DP","id"], axis=1, inplace=True)
print(dupes1.columns)

Index(['belongs_to_collection', 'budget', 'genres', 'homepage', 'imdb_id',
       'original_language', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'title', 'vote_average', 'vote_count',
       'release_year', 'imdb_id_str_OR'],
      dtype='object')


In [30]:
# create multi-level index to sort dupes and compare values
# Results: all of the data on the duplicates records match, except for
# a couple of minor differences on vote counts
# more of the duplicates have differences in the popularity rating
# but these two variables alone, do not account for all the duplicates.
# use df.drop_duplicates(keep=first)
dupes2=dupes1.copy()
# name row index to make it easy to reference?
dupes2["o_index"] = dupes2.index.copy()
dupes2["o_index"]
dupes2.columns
dupes2=dupes2.sort_values(by=["imdb_id_str_OR", "o_index"])
print(dupes2)

                                   belongs_to_collection    budget  \
15702                                                NaN         0   
29374                                                NaN         0   
19890                                                NaN         0   
21165                                                NaN         0   
5710                                                 NaN         0   
20899                                                NaN         0   
13603                                                NaN         0   
28860                                                NaN         0   
9576                                                 NaN  10000000   
26625                                                NaN  10000000   
676                                                  NaN         0   
1465                                                 NaN         0   
5130                                                 NaN  26000000   
33743               

In [31]:
# Return to movies_yr_df and drop duplicates
print(f'number of movies before dedupe: {len(movies_year_df)}')
movies_year_df.drop_duplicates(subset=["imdb_id_str"], keep='first', inplace=True)
print(f'number of movies after dedupe: {len(movies_year_df)}')

number of movies before dedupe: 37073
number of movies after dedupe: 37049


In [33]:
# Identify duplicates in IMDb_gt1969_df
IMDb_gt1969_df["imdb_id_str"]=IMDb_gt1969_df["imdb_id"].apply(lambda z: z.strip())
IMDb_gt1969_df["imdb_id_str"].duplicated(keep=False).sum()

0

In [34]:
merge_movies_df = pd.merge(IMDb_gt1969_df, movies_year_df, on='imdb_id_str', suffixes=['_TM','_MD'], indicator=True, validate="1:1")
print(merge_movies_df.describe())

           duration      avg_vote         votes     metascore  \
count  26975.000000  26975.000000  2.697500e+04  10239.000000   
mean     102.680630      6.116816  2.580955e+04     55.180975   
std       19.620413      1.107370  9.044864e+04     17.506842   
min       41.000000      1.100000  1.000000e+02      1.000000   
25%       90.000000      5.500000  7.950000e+02     43.000000   
50%       99.000000      6.300000  2.466000e+03     56.000000   
75%      110.000000      6.900000  1.109300e+04     68.000000   
max      442.000000      9.300000  2.278845e+06    100.000000   

       reviews_from_users  reviews_from_critics  release_year_TM  \
count        26277.000000          26017.000000     26975.000000   
mean            90.730182             54.728793      2000.982243   
std            230.504501             82.871006        13.026370   
min              1.000000              1.000000      1970.000000   
25%              8.000000              9.000000      1992.000000   
50%   

In [35]:
print(merge_movies_df.columns)

Index(['imdb_id_TM', 'title_TM', 'original_title', 'year', 'date_published',
       'genre', 'duration', 'country', 'language', 'director', 'writer',
       'production_company', 'actors', 'description', 'avg_vote', 'votes',
       'budget_TM', 'usa_gross_income', 'worlwide_gross_income', 'metascore',
       'reviews_from_users', 'reviews_from_critics', 'release_year_TM',
       'imdb_id_str', 'belongs_to_collection', 'budget_MD', 'genres',
       'homepage', 'id', 'imdb_id_MD', 'original_language', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'title_MD',
       'vote_average', 'vote_count', 'release_year_MD', '_merge'],
      dtype='object')
